In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torchvision.transforms import functional as F
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio as psnr

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
class NIMAModel(nn.Module):
    def __init__(self):
        super(NIMAModel, self).__init__()
        self.base_model = models.mobilenet_v2(weights='IMAGENET1K_V2')
        self.base_model.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.base_model.last_channel, 10)
        )
    
    def forward(self, x):
        return self.base_model(x)

nima_model = NIMAModel().to(device)
nima_model.eval()

def calculate_nima_score(image_tensor):
    with torch.no_grad():
        scores = nima_model(image_tensor.unsqueeze(0).to(device))
        scores = torch.softmax(scores, dim=1)
        return torch.sum(scores * torch.arange(1, 11).to(device)).item()


In [ ]:
import sys
import os

# Forcer l'installation de nbformat dans l'environnement actuel du notebook
!{sys.executable} -m pip install --upgrade nbformat

import nbformat
import zipfile

def clear_notebook_output(notebook_path, output_path=None):
    """
    Nettoyer toutes les sorties d'un notebook Jupyter pour le rendre plus léger.
    """
    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)


    for cell in notebook.cells:
        if 'outputs' in cell:
            cell['outputs'] = []
        if 'execution_count' in cell:
            cell['execution_count'] = None


    output_path = output_path or notebook_path
    with open(output_path, 'w', encoding='utf-8') as f:
        nbformat.write(notebook, f)
    print(f"Nettoyage terminé ! Notebook allégé sauvegardé sous : {output_path}")

def zip_notebook(notebook_path):
    """
    Compresser le notebook nettoyé pour réduire sa taille.
    """
    zip_path = notebook_path.replace(".ipynb", ".zip")
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(notebook_path)
    print(f"Notebook compressé et sauvegardé sous : {zip_path}")

def delete_large_files():
    """
    Supprimer tous les fichiers volumineux tels que .pth, .h5 et les checkpoints.
    """
    for root, dirs, files in os.walk("."):
        for file in files:
            if file.endswith(".pth") or file.endswith(".h5"):
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print(f"Fichier supprimé : {file_path}")


    if os.path.exists('.ipynb_checkpoints'):
        os.system("rm -rf .ipynb_checkpoints")
        print("Checkpoints supprimés.")

    print("Nettoyage des fichiers volumineux terminé !")

notebook_name = "CGNet_BSD500_complet.ipynb"


clear_notebook_output(notebook_name, "notebook_allégé.ipynb")
zip_notebook("notebook_allégé.ipynb")
delete_large_files()

print("\n📦 Opérations terminées ! Le notebook a été nettoyé, compressé et les fichiers inutiles supprimés.")


In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio as psnr
from scipy.ndimage import median_filter
from bm3d import bm3d


# Configuration du périphérique
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Dataset pour le débruitage
class DenoisingDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.image_paths = [
            os.path.join(root_dir, img) for img in os.listdir(root_dir)
            if img.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))
        ]
        if not self.image_paths:
            raise ValueError(f"Aucune image valide trouvée dans le répertoire: {root_dir}")
        self.transform = transform

    def add_gaussian_noise(self, image, mean=0, std=0.3):
        noise = torch.randn_like(image) * std + mean
        return torch.clamp(image + noise, 0, 1)

    def add_salt_pepper_noise(self, image, prob=0.08):
        noisy_image = image.clone()
        num_pixels = image.shape[1] * image.shape[2]
        num_pepper = int(prob * num_pixels * 0.5)
        pepper_coords = (torch.randint(0, image.shape[1], (num_pepper,)),
                         torch.randint(0, image.shape[2], (num_pepper,)))
        noisy_image[:, pepper_coords[0], pepper_coords[1]] = 0
        num_salt = int(prob * num_pixels * 0.5)
        salt_coords = (torch.randint(0, image.shape[1], (num_salt,)),
                       torch.randint(0, image.shape[2], (num_salt,)))
        noisy_image[:, salt_coords[0], salt_coords[1]] = 1
        return noisy_image

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        # Ajout de bruit (aléatoire)
        if np.random.rand() > 0.5:
            noisy_image = self.add_gaussian_noise(image)
        else:
            noisy_image = self.add_salt_pepper_noise(image)

        return noisy_image, image.float()  # Retourne l'image bruitée et l'image propre


# Transformations pour le dataset
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Chargement du dataset
train_dataset = DenoisingDataset(root_dir='archive/images/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)


# Définition complète du modèle CGNet
class ContextGuidedBlock(nn.Module):
    def __init__(self, nIn, nOut, dilation_rate=2):
        super(ContextGuidedBlock, self).__init__()
        self.conv1x1 = nn.Conv2d(nIn, nOut // 2, kernel_size=1)
        self.conv3x3 = nn.Conv2d(nOut // 2, nOut, kernel_size=3, padding=dilation_rate, dilation=dilation_rate)
        self.bn = nn.BatchNorm2d(nOut)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv1x1(x)
        x = self.conv3x3(x)
        x = self.bn(x)
        return self.relu(x)


class AttentionBlock(nn.Module):
    def __init__(self, nOut):
        super(AttentionBlock, self).__init__()
        self.attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(nOut, nOut // 16, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(nOut // 16, nOut, kernel_size=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        attention_map = self.attention(x)
        return x * attention_map


class CGNet(nn.Module):
    def __init__(self):
        super(CGNet, self).__init__()
        self.block1 = ContextGuidedBlock(3, 32)
        self.block2 = ContextGuidedBlock(32, 64)
        self.attention_block = AttentionBlock(64)
        self.conv_final = nn.Conv2d(64, 3, kernel_size=1)

    def forward(self, x):
        x1 = self.block1(x)
        x2 = self.block2(x1)
        x3 = self.attention_block(x2)
        output = self.conv_final(x3)
        return output, x3


# Entraînement avec prétraitement utilisé uniquement pour les métriques
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for noisy_imgs, clean_imgs in tqdm(train_loader):
        noisy_imgs = noisy_imgs.to(device).float()
        clean_imgs = clean_imgs.to(device).float()

        optimizer.zero_grad()
        outputs, _ = model(noisy_imgs)

        # Calcul de la perte directement sur les sorties brutes
        loss = criterion(outputs, clean_imgs)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

    # Prétraitement et PSNR (uniquement pour l'affichage ou l'évaluation)
    model.eval()
    with torch.no_grad():
        for noisy_imgs, clean_imgs in train_loader:
            noisy_imgs = noisy_imgs.to(device).float()
            clean_imgs = clean_imgs.to(device).float()
            outputs, _ = model(noisy_imgs)

            # Appliquer le prétraitement sur l'image débruitée
            preprocessed_images = []
            for output in outputs:
                output_np = output.detach().permute(1, 2, 0).cpu().numpy()
                denoised_image = median_filter(output_np, size=3)  # Filtre médian
                preprocessed_image = torch.tensor(denoised_image).permute(2, 0, 1).to(device)
                preprocessed_images.append(preprocessed_image)

            preprocessed_images = torch.stack(preprocessed_images)
            psnr_denoised = psnr(clean_imgs.cpu().numpy(), preprocessed_images.cpu().numpy())
            print(f"PSNR Denoised: {psnr_denoised:.2f}")
            break  # Affichage d'un seul batch pour chaque epoch


# Sauvegarde du modèle
torch.save(model.state_dict(), 'cgnet_full_preprocessed.pth')
print("Modèle sauvegardé.")





# Affichage des échantillons
def show_sample_with_metrics(model, dataset, index):
    """
    Affiche les images bruitées, originales et débruitées avec PSNR.
    """
    model.eval()
    noisy_img, clean_img = dataset[index]
    noisy_img = noisy_img.unsqueeze(0).to(device)
    clean_img = clean_img.unsqueeze(0).to(device)

    with torch.no_grad():
        output, _ = model(noisy_img)
        output = output.cpu().squeeze()

    psnr_noisy = psnr(clean_img.squeeze().permute(1, 2, 0).cpu().numpy(), noisy_img.squeeze().permute(1, 2, 0).cpu().numpy())
    psnr_denoised = psnr(clean_img.squeeze().permute(1, 2, 0).cpu().numpy(), output.permute(1, 2, 0).numpy())

    print(f"PSNR Noisy: {psnr_noisy:.2f}, PSNR Denoised: {psnr_denoised:.2f}")

    plt.figure(figsize=(18, 6))
    plt.subplot(1, 3, 1)
    plt.title("Noisy Image")
    plt.imshow(noisy_img.squeeze().permute(1, 2, 0).cpu().numpy())
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.title("Clean Image")
    plt.imshow(clean_img.squeeze().permute(1, 2, 0).cpu().numpy())
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.title("Denoised Image")
    plt.imshow(output.permute(1, 2, 0).cpu().numpy())
    plt.axis('off')

    plt.show()


show_sample_with_metrics(model, train_dataset, 0)
show_sample_with_metrics(model, train_dataset, 1)
show_sample_with_metrics(model, train_dataset, 2)
show_sample_with_metrics(model, train_dataset, 3)


In [ ]:
# Exemple d'affichage
show_sample_with_metrics(model, train_dataset, 0)
show_sample_with_metrics(model, train_dataset, 1)
show_sample_with_metrics(model, train_dataset, 2)
show_sample_with_metrics(model, train_dataset, 3)
show_sample_with_metrics(model, train_dataset, 4)
show_sample_with_metrics(model, train_dataset, 5)
show_sample_with_metrics(model, train_dataset, 6)
show_sample_with_metrics(model, train_dataset, 7)
show_sample_with_metrics(model, train_dataset, 8)
show_sample_with_metrics(model, train_dataset, 9)
show_sample_with_metrics(model, train_dataset, 10)
show_sample_with_metrics(model, train_dataset, 11)
show_sample_with_metrics(model, train_dataset, 12)
show_sample_with_metrics(model, train_dataset, 13)
show_sample_with_metrics(model, train_dataset, 14)
show_sample_with_metrics(model, train_dataset, 15)